In [1]:
import numpy as np
from scipy.stats import norm

In [4]:
import numpy as np

def black_scholes_fd(S0, K, T, r, sigma, M, N, Smax, option_type="call"):
    """
    Solves the Black-Scholes equation for European Call or Put options
    using the finite difference method.
    
    Parameters:
        S0: Initial price of the underlying asset
        K: Strike price
        T: Time to maturity (in years)
        r: Risk-free rate
        sigma: Volatility
        M: Number of time steps
        N: Number of asset price steps
        Smax: Maximum asset price
        option_type: "call" or "put" (default: "call")
    
    Returns:
        A 2D array containing the option values on the grid.
    """
    # Discretization
    dt = T / M
    dS = Smax / N
    
    # Create grid for S and t
    S = np.linspace(0, Smax, N+1)
    t = np.linspace(0, T, M+1)
    
    # Initialize option price grid
    V = np.zeros((M+1, N+1))
    
    # Set terminal condition (payoff at maturity)
    if option_type == "call":
        V[-1, :] = np.maximum(S - K, 0)  # Payoff for call option
    elif option_type == "put":
        V[-1, :] = np.maximum(K - S, 0)  # Payoff for put option
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")
        
  
    if option_type == "call":
        V[:, -1] = Smax - K  # For call, it's Smax - K
    elif option_type == "put":
        V[:, -1] = K * np.exp(-r * t) - Smax  # For put, it's K * exp(-rT) - Smax

    V[:, 0] = 0 
    # Backward iteration through time
    for n in range(M-1, -1, -1):  # Loop over time steps (from maturity to present)
        for i in range(1, N):  # Loop over spatial points (excluding boundaries)
            # Calculate second spatial derivative (gamma)
            gamma = (V[n+1, i+1] - 2 * V[n+1, i] + V[n+1, i-1]) / (dS**2)
            # Calculate first spatial derivative (delta)
            delta = (V[n+1, i+1] - V[n+1, i-1]) / (2 * dS)

            # Update option value using the finite difference scheme
            V[n, i] = V[n+1, i] + dt * (
                0.5 * sigma**2 * S[i]**2 * gamma + r * S[i] * delta - r * V[n+1, i]
            )

    return V, S

# Parameters
S0 = 100   # Initial price of the underlying asset
K = 110    # Strike price
T = 0.5    # Time to maturity (in years)
r = 0.02    # Annual interest rate
sigma = 0.2 # Volatility
M = 500    # Number of time intervals
N = 100    # Number of asset price intervals
Smax = 300 # Maximum asset price

# Solve for call and put options
V_call, S_call = black_scholes_fd(S0, K, T, r, sigma, M, N, Smax, "call")
V_put, S_put = black_scholes_fd(S0, K, T, r, sigma, M, N, Smax, "put")

# Find the option price for S0
call_price = np.interp(S0, S_call, V_call[0, :])
put_price = np.interp(S0, S_put, V_put[0, :])

print(f"Call Option Price: {call_price:.2f}")
print(f"Put Option Price: {put_price:.2f}")

Call Option Price: 2.50
Put Option Price: 11.40
